# Python Lab 1

到這裡為止，就是大致上的Python的基礎運用

下面我們就以實際例子來做幾個 Lab

## Lab 1

在 `file` 下有個 `pwd_list.xlsx`

裡面大概是這樣

| account | jdbc_string | password | base64_pwd
| -- | -- | -- | --
P_LOAD_CSP | jdbc:teradata://10.68.64.168/TMODE=TERA,CHARSET=ASCII,CLIENT_CHARSET=MS950,LOB_SUPPORT=OFF | 50D85DAC17DFE8F9C868BB588DD3799F | b2RzcHN0YWdlY3NwIA==
DAS2CRMODS | jdbc:sqlserver://10.140.0.2:1433;useUnicode=true;characterEncoding=UTF-8 | A9E27EB342A4852D08AEB6EC9CEB4DF9 | disxMjM0NTY=


這個檔案是 Trinity 的 Connection XML 解析出來的欄位

其中包含

1. Account 帳號
2. JDBC String
3. Password (編碼過)
4. Base64_pwd (以 Base64 編碼的密碼)


<b><font size=4> `請把 Base64_pwd 欄位的所有值解碼回正常的密碼` </font></b>


````{note}
Base64 編碼是可以被解碼的

Python 自帶的 Lib 中，有 Base64編、解碼的 Library

```{caution}
使用上請注意 Base64 解碼與編碼， 都必須使用位元 （Byte Object）
```
````

### Lab 1 參考

In [1]:
import pandas as pd
import base64


df = pd.read_excel('file/pwd_list.xlsx', engine='openpyxl')

df['real_pwd'] = df['base64_pwd'].apply(lambda x: base64.b64decode(x.encode('ascii')).decode('ascii'))

df.to_excel('file/pwd_list_ANSWER.xlsx', index=False)

## Lab 1.5

你如果想問

啊 ！ password 這個欄位資料要做什麼用？

<s> 現在要用到了 </s>

你可以在 `./bin/` 下找到 `decode.jar` 這是一個解碼的 JAR 檔

<b><font size=4> `現在，請把 password 所有值，當成參數給這個 JAR 來執行，並再輸出一個報表` </font></b>

我們這邊要透過 Subprocess 來使用 JAR 檔，並透過 Standard Output 來交換資料

````{admonition} Subprocess Library
Python 有內建 Subprocess Library

並且可以重新導向 `Standard Output` & `Standard Error`

直接使用（將 External Command 拆成數個 String）

```python
import subprocess


command = ['java',
           '-jar',
           './bin/decode.jar',
           'F3A5806FF36F36D96C1F8E4310F6ECC0'
           ]
           
process = subprocess.Popen(command , universal_newlines=True, stdout=subprocess.PIPE, bufsize=1)

std = process.communicate()

std

````

```{caution}
你應該有發現，不論是 Base64 Password 和 Password 都有重複值的問題吧！

因為我們接下來要透過 Subprocess Call Java

我們應該要先 Distinct Password 來增加效能 ！
```

### Lab 1.5 參考

````{admonition} 請參考
:class: dropdown

```python
import subprocess


df = pd.read_excel('file/pwd_list.xlsx', engine='openpyxl')

pwd_set = set()
pwd_dict = {}

for pwd in df['password']:
    pwd_set.add(pwd)

for pwd in pwd_set:
    
    command = ['java',
               '-jar',
               './bin/decode.jar',
               '{}'.format(pwd)
               ]
    print('現在解碼 : {}'.format(pwd))
    
    process = subprocess.Popen(command , universal_newlines=True, stdout=subprocess.PIPE, bufsize=1)

    std = process.communicate()
    
    pwd_dict[pwd] = std[0].replace('\n', '')



df['java_pwd'] = df['password'].apply(lambda x: pwd_dict.get(x))

df.to_excel('file/pwd_list_ANSWER.xlsx', index=False)
```
````